In [1]:
import hopsworks
from datetime import datetime,timedelta
import pandas as pd
import optuna
from sklearn.metrics import mean_absolute_error
import joblib
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

from src.data import *
from src.config import *
from src.data_split import *
from src.model import *

#### 1. Hopsworks feature store

In [2]:
### Connecting to Hopsworks Feature Store


hw_project = hopsworks.login(project=HOPSWORKS_PROJECT,api_key_value=HOPSWORKS_API_KEY)
fs = hw_project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/467093
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
### Connect to the Feature Group

fg = fs.get_or_create_feature_group(name=FEATURE_GROUP_NAME,
                             description='Hourly data of taxi demand in NYC in TimeSeries format',
                             version=FEATURE_GROUP_VERSION,
                             primary_key=['pickup_hour','pickup_location_id'],
                             event_time='pickup_hour')

In [4]:
### Create Feature View

try:
    fs.create_feature_view(name=FEATURE_VIEW_NAME,
                           version=FEATURE_VIEW_VERSION,
                           query=fg.select_all())
except:
    print("Feature view already exists")

Feature view already exists


In [5]:
### Connect to Feature View

fv = fs.get_feature_view(name=FEATURE_VIEW_NAME, version=FEATURE_VIEW_VERSION)

In [10]:
%%time

### Get Training data

taxi_data_ts,_ = fv.training_data(description="Hourly taxi rides - Time Series data")

Finished: Reading data from Hopsworks, using ArrowFlight (35.99s) 
CPU times: user 1.11 s, sys: 993 ms, total: 2.1 s
Wall time: 38.8 s


In [11]:
taxi_data_ts.sort_values(by=['pickup_hour', 'pickup_location_id'],inplace=True)
taxi_data_ts.columns = ['pickup_time','pickup_location','count_pickup_loc']
taxi_data_ts.reset_index(drop=True,inplace=True)
taxi_data_ts.to_parquet(TRANSFORMED_PATH + "rides.parquet") #compression='snappy', index=None   

#### 2. Transform Time Series data into Tabular Data (Features, Target)

In [12]:
%%time

window_size = 672 #1 month i.e., 28 days => 28*24 hours = 672
step_size = 23

features,target = transform_timeseriesdata_into_features_target(window_size,step_size)
print("Features : ",features.shape,"Target : ",target.shape)

Features :  (191462, 674) Target :  (191462, 1)
CPU times: user 50.5 s, sys: 2.25 s, total: 52.8 s
Wall time: 53.7 s


In [13]:
df = features
df['target_rides_next_hour'] = target
df['pickup_hour'] = pd.to_datetime(df['pickup_hour']).dt.tz_convert(None)
df

,rides_previous_672_hours,rides_previous_671_hours,rides_previous_670_hours,rides_previous_669_hours,rides_previous_668_hours,rides_previous_667_hours,rides_previous_666_hours,rides_previous_665_hours,rides_previous_664_hours,rides_previous_663_hours,...,rides_previous_7_hours,rides_previous_6_hours,rides_previous_5_hours,rides_previous_4_hours,rides_previous_3_hours,rides_previous_2_hours,rides_previous_1_hours,pickup_hour,pickup_location_id,target_rides_next_hour
0,0,0,0,0,1,1,0,2,0,0,...,2,0,1,0,0,0,0,2022-01-29 00:00:00,1,0
1,0,0,0,0,0,0,4,1,2,1,...,0,0,0,0,0,1,0,2022-01-29 23:00:00,1,0
2,0,0,0,0,0,0,0,0,0,2,...,2,2,0,1,2,0,0,2022-01-30 22:00:00,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,1,2,1,0,1,2022-01-31 21:00:00,1,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,2022-02-01 20:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191457,4,2,6,2,8,2,4,0,0,0,...,4,0,0,2,6,4,10,2024-02-18 07:00:00,265,26
191458,14,3,4,8,5,2,4,1,0,2,...,11,10,0,0,2,2,3,2024-02-19 06:00:00,265,12
191459,4,8,3,1,2,2,6,7,5,7,...,17,11,6,0,2,3,3,2024-02-20 05:00:00,265,3
191460,4,2,7,5,3,3,6,3,9,15,...,11,4,7,1,4,1,1,2024-02-21 04:00:00,265,1


#### 3. Train - Test split

- We have data from 2022, 2023 until November, 2024..
- Last 4 months (16+1 = 17 weeks) can be Test data

In [14]:
cutoff_dt = datetime.now().replace(minute=0,second=0,microsecond=0) - timedelta(weeks=17)
X_train,y_train,X_test,y_test = train_test_split(df,cutoff_dt)

print("Training set : ",X_train.shape,y_train.shape)
print("Test set : ",X_test.shape,y_test.shape)

Training set :  (173968, 674) (173968,)
Test set :  (17494, 674) (17494,)


#### 4. Hyperparameter tuning using Optuna

In [15]:
# Create an objective function that Optuna tries to minimize

def objective(trial):
    
    r_params = {
        "metric": "mae",
        "verbosity": -1,
        "num_leaves":trial.suggest_int('num_leaves',2,256), 
        "bagging_fraction":trial.suggest_float('bagging_fraction',0.2,1.0),
        "feature_fraction":trial.suggest_float('feature_fraction',0.2,1.0),
        "min_data_in_leaf":trial.suggest_int('min_data_in_leaf',3,100)
    }
    
    pipe = get_pipeline(r_params)
    pipe.fit(X_train,y_train)
    
    y_pred_lgb_ht = pipe.predict(X_test)
    error_metric_lgb_ht = mean_absolute_error(y_test,y_pred_lgb_ht)
    print("{:.4f}".format(error_metric_lgb_ht))
    
    return error_metric_lgb_ht

In [16]:
# Run the optimization (i.e., minimization here)

study = optuna.create_study(study_name='lgb_nyc_study',direction='minimize')
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2024-02-22 13:49:56,005] A new study created in memory with name: lgb_nyc_study


  0%|          | 0/10 [00:00<?, ?it/s]

3.2371
[I 2024-02-22 13:50:24,452] Trial 0 finished with value: 3.2370942062936594 and parameters: {'num_leaves': 12, 'bagging_fraction': 0.3087492117003085, 'feature_fraction': 0.9155945476716842, 'min_data_in_leaf': 19}. Best is trial 0 with value: 3.2370942062936594.
2.7835
[I 2024-02-22 13:51:03,440] Trial 1 finished with value: 2.7835231199837365 and parameters: {'num_leaves': 185, 'bagging_fraction': 0.5229836000412156, 'feature_fraction': 0.7088243865303268, 'min_data_in_leaf': 91}. Best is trial 1 with value: 2.7835231199837365.
2.7196
[I 2024-02-22 13:51:44,562] Trial 2 finished with value: 2.719646405252628 and parameters: {'num_leaves': 239, 'bagging_fraction': 0.7431273470513295, 'feature_fraction': 0.7253417414947081, 'min_data_in_leaf': 63}. Best is trial 2 with value: 2.719646405252628.
3.1990
[I 2024-02-22 13:52:15,307] Trial 3 finished with value: 3.1989853352344575 and parameters: {'num_leaves': 132, 'bagging_fraction': 0.9305500000063383, 'feature_fraction': 0.209657

In [17]:
print('Best parameters', study.best_params)
print('Best score', study.best_value)
print('Best model', study.best_trial)

Best parameters {'num_leaves': 239, 'bagging_fraction': 0.7431273470513295, 'feature_fraction': 0.7253417414947081, 'min_data_in_leaf': 63}
Best score 2.719646405252628
Best model FrozenTrial(number=2, state=TrialState.COMPLETE, values=[2.719646405252628], datetime_start=datetime.datetime(2024, 2, 22, 13, 51, 3, 443417), datetime_complete=datetime.datetime(2024, 2, 22, 13, 51, 44, 561872), params={'num_leaves': 239, 'bagging_fraction': 0.7431273470513295, 'feature_fraction': 0.7253417414947081, 'min_data_in_leaf': 63}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.2, step=None), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.2, step=None), 'min_data_in_leaf': IntDistribution(high=100, log=False, low=3, step=1)}, trial_id=2, value=None)


#### 5. Re-fit the model using the best parameters found

In [18]:
pipe = get_pipeline(study.best_params)
pipe.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.7253417414947081, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7253417414947081
[LightGBM] [Warning] bagging_fraction is set=0.7431273470513295, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7431273470513295
[LightGBM] [Warning] min_data_in_leaf is set=63, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=63
[LightGBM] [Warning] feature_fraction is set=0.7253417414947081, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7253417414947081
[LightGBM] [Warning] bagging_fraction is set=0.7431273470513295, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7431273470513295
[LightGBM] [Warning] min_data_in_leaf is set=63, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=63
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.487652 seconds.
You can set `force_row_wise=true` to remove the ov

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function avg_rides_last4weeks at 0x131416430>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function extract_features_pickuptime at 0x1314164c0>)),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.7431273470513295,
                               feature_fraction=0.7253417414947081,
                               min_data_in_leaf=63, num_leaves=239))])

In [19]:
y_pred_lgb_ht = pipe.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.7253417414947081, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7253417414947081
[LightGBM] [Warning] bagging_fraction is set=0.7431273470513295, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7431273470513295
[LightGBM] [Warning] min_data_in_leaf is set=63, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=63


#### 6. Evaluate model

In [20]:
error_metric_lgb_ht = mean_absolute_error(y_test,y_pred_lgb_ht)
print("{:.4f}".format(error_metric_lgb_ht))

2.7196


#### 7. Save model

In [21]:
joblib.dump(pipe,MODEL_PATH+'/nyc_taxi_pipe_model.pkl')

['/Users/reshma/AI/MLOPS Project/taxi_demand_predictor/model/nyc_taxi_pipe_model.pkl']

#### 8. Push model to Model Registry in Hopsworks

In [23]:
hw_project = hopsworks.login(project=HOPSWORKS_PROJECT,api_key_value=HOPSWORKS_API_KEY)
mr = hw_project.get_model_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/467093
Connected. Call `.close()` to terminate connection gracefully.


In [22]:
input_schema = Schema(X_train)
output_schema = Schema(y_train)
ms = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [24]:
taxi_model = mr.sklearn.create_model(name="taxi_demand_predictor_next_hour",
                                      metrics={'test_mae': error_metric_lgb_ht},
                                      description="LightGBM regressor with hyperparameter tuning",
                                      input_example=X_train.sample(), 
                                      model_schema=ms)
taxi_model.save(MODEL_PATH+'/nyc_taxi_pipe_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/2181427 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/2073 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/57755 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/467093/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)